# Lademanagementsimulation   


### Simulations Parameter

In [1]:
solarpeakleistung = 125 #Einheit Kilowattpeak (kWP)
anzahl_bevs_pro_tag = 100
ladeleistung_pro_bev = 11 #Einheit kW
maximale_ladezeit = 30 #Einheit Minuten

### Simulations Eingangsdaten

In [2]:
import simulationInputDataVisualisation
simulationInputDataVisualisation.create_jupyter_dash_app(solarpeakleistung)

### Verteilungsalgorithmus Polling - FIFO
- der Algorithmus weist den BEVs jeweils ein Zustand zu, die Zustände sind wartend (BEV parkt und will laden), ladend (BEV wird mit Solarenergie geladen), nicht parkend (BEV befindet sich nicht im Lademanagmentsystem)
- pro BEV Ladeleistung von z.B. 2kW (ladeleistung_pro_bev), wenn verfügbarer Solarstrom > Ladeleistung, dann wird der Solarstrom auf mehrere BEVs verteilt
- aktuell wartende Verbraucher werden in einer Warteliste (Pollingliste) abgearbeitet, nach einem variablen Zeitintervall (maximale_ladezeit, z.B. 30min) wird Verbraucher wieder hinten an die Warteliste angefügt. 
- dabei wird immer das erste BEV der Warteliste auf die Ladeliste genommen, unabhängig von der Länge der Parkzeit
- Wenn im Tagesverlauf die Anzahl bereits ladender BEVs größer wird als die Anzahl möglich ladender BEVs, wird das erste BEV der Ladeliste wieder nach hinten an die Warteliste gesetzt

### Verteilungsalgorithmus Polling - mit gleichmäßiger Verteilung
- wie FIFO Abarbeitung der BEVs auf der Warteliste
- allerdings werden BEVs nach dem sie einmal aufgeladen wurden auf eine neue Warteliste gesetzt mit geringerer Priorität => so soll gewährleistet werden, dass erstmal alle BEVs einmal aufgeladen werden, bevor BEVs mehrmals geladen werden 

### Prognose (Berücksichtigung der Parkdauer - Optimale Ausnutzung)
- Ziel ist das möglichst alle gleich viel bekommen und der Solarstrom optimal ausgenutzt wird ⇒ nichts geht verloren
- Berechnung des Ladeintervalls:
    - Wann sind während des angegebenen Parkintervalls am wenigsten andere BEVs am Warten? (Minimum Anzahl wartender BEVs in Parkintevall)
    - Ladestart: sobald Minimum Anzahl wartender BEVs in Parkintevall erreicht wird
    - Ladeende: nach fester Zeit oder nachdem Anzahl wartender BEVs nicht mehr Minimum Anzahl wartender BEVs entspricht
- Solarleistung wird zu jedem Zeitpunkt gleichmäßig unter ladenden BEVs verteilt, also Ladeleistung = Verfügbare Solarleistung / Ladende BEVs

### Prognose (Berücksichtigung der Parkdauer - Belohnung)
- Ziel: länger parkende BEVs sollen belohnt werden und mehr Solarenergie garantiert bekommen
- Möglichkeiten: mehr Solarenergie oder längeres Ladeintervall

In [3]:
import distributionAlgorithmPolling
import simulationClasses
import simulationData
import simulationOutputDataVisualisation
import distributionAlgorithmForecast

simulation_day = simulationClasses.SimulationDay(anzahl_bevs_pro_tag)
bev_data = simulationData.BevData()
table_dict = simulationData.TableDict()
simulation_data = simulationData.SimulationData()

# Auswahl des Algorithmus
algorithm = distributionAlgorithmPolling.Algorithm.POLLING_FIFO
# algorithm = distributionAlgorithmPolling.Algorithm.POLLING_EVEN_DISTRIBUTION
# distributionAlgorithmPolling.start_simulation(solarpeakleistung, maximale_ladezeit, ladeleistung_pro_bev, simulation_day, bev_data, table_dict, simulation_data, algorithm)

distributionAlgorithmForecast.start_algorithm(simulation_data, simulation_day, maximale_ladezeit, solarpeakleistung, 
                    bev_data, table_dict, ladeleistung_pro_bev)

{0: [(8.0, 6.1), 'nicht parkend', [(699, 30, 0)]], 1: [(12.5, 3.9), 'wartend', [(735, 30, 0)]], 2: [(12.5, 4.9), 'wartend', [(831, 30, 0)]], 3: [(13.0, 5.6), 'wartend', [(832, 30, 0)]], 4: [(8.0, 6.5), 'nicht parkend', [(693, 30, 0)]], 5: [(13.5, 6.4), 'wartend', [(833, 30, 0)]], 6: [(8.5, 5.5), 'nicht parkend', [(694, 30, 0)]], 7: [(8.5, 6.3), 'nicht parkend', [(695, 30, 0)]], 8: [(9.0, 6.3), 'nicht parkend', [(696, 30, 0)]], 9: [(9.0, 3.4), 'nicht parkend', [(700, 30, 0)]], 10: [(8.5, 5.8), 'nicht parkend', [(701, 30, 0)]], 11: [(8.5, 4.7), 'nicht parkend', [(702, 30, 0)]], 12: [(8.0, 6.3), 'nicht parkend', [(703, 30, 0)]], 13: [(9.0, 8.0), 'wartend', [(704, 30, 0)]], 14: [(14.5, 7.7), 'wartend', [(855, 30, 0)]], 15: [(12.0, 5.1), 'wartend', [(711, 30, 0)]], 16: [(11.5, 4.6), 'wartend', [(697, 30, 0)]], 17: [(8.0, 4.7), 'nicht parkend', [(698, 30, 0)]], 18: [(8.5, 7.7), 'wartend', [(687, 30, 0)]], 19: [(12.0, 5.3), 'wartend', [(712, 30, 0)]], 20: [(8.5, 5.2), 'nicht parkend', [(688, 

### Ergebnis Algorithmus

In [4]:
simulationOutputDataVisualisation.print_algorithm_result(bev_data, simulation_data, anzahl_bevs_pro_tag)

>**Ergebnis**<br>Geladene BEVs: 0 von 100<br>Unterbrochene Aufladevorgänge 0<br>Aufgeladene Solarenergie insgesamt: 0 kWh<br>Ungenutzte Solarenergie insgesamt: 0 kWh

### Tabellarische Übersicht Park- und Ladeverlauf der BEVs

In [ ]:
simulationOutputDataVisualisation.create_tabular_overview_per_minute_slider(table_dict)

interactive(children=(IntSlider(value=480, description='Minute: ', max=960, min=480), Output()), _dom_classes=…

In [6]:
import figureGeneration

figureGeneration.create_charging_power_figure(simulation_day)

In [7]:
figureGeneration.create_bev_number_figure(simulation_data)